In [ ]:
from __future__ import division, print_function
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Image, display, clear_output
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets
import tensorflow as tf
import math
from tensorflow.python.framework.ops import reset_default_graph

from sklearn.utils import shuffle

In [ ]:
# Load data (download if you haven't already)
from tensorflow.examples.tutorials.mnist import input_data
mnist_data = input_data.read_data_sets('MNIST_data', 
                                       one_hot=True,   # Convert the labels into one hot encoding
                                       dtype='float32', # rescale images to `[0, 1]`
                                       reshape=False, # Don't flatten the images to vectors
                                      )

gpu_opts = tf.GPUOptions(per_process_gpu_memory_fraction=0.2)


In [ ]:
from tensorflow import layers
from tensorflow.contrib.layers import fully_connected, convolution2d, convolution2d_transpose, batch_norm, max_pool2d, dropout
from tensorflow.python.ops.nn import relu, elu, relu6, sigmoid, tanh, softmax, softplus

In [ ]:
# reset graph
reset_default_graph()

# -- THE MODEL --#
num_channels = 1; #Black and white for MNIST
num_classes = 1
k = 16;
height = width = 28

# Layer definitions
def layer(x, units):
    with tf.name_scope('layer_' + str(units)):
        x = fully_connected(x, num_outputs=units, activation_fn=relu,
                             normalizer_fn=batch_norm)
        x = convolution2d(x, num_outputs=units, kernel_size=(3, 3),
                                 stride=1)
        return dropout(x, is_training=is_training_pl)
    
def dense_block(x, num_layers):
    with tf.name_scope('dense_' + str(num_layers)):
        res = []
        for i in range(num_layers):
            layer_output = layer(x, k)
            x = tf.concat([x, layer_output], axis=-1)
            res.append(layer_output)
        return x, res
    

def transition_up(x, units):
    return convolution2d_transpose(x, num_outputs=units, kernel_size=(3, 3), stride=2)
    
    
def transition_down(x, units, pooling=True):
    #x = batch_norm(x, scope='tdown_batch_norm') #Batch norm should be included in fully_connected layer below
    x = fully_connected(x, num_outputs=units, activation_fn=relu,
                     normalizer_fn=batch_norm)
    x = convolution2d(x, num_outputs=units, kernel_size=(1, 1),
                         stride=1)
    x = dropout(x, is_training=is_training_pl)
    if pooling:
        x = max_pool2d(x, kernel_size=(2, 2))
    return x

# - Tiramisu Architecture - #
# Input placeholder
x_pl = tf.placeholder(tf.float32, [None, height, width, num_channels], 'x_pl')
y_pl = tf.placeholder(tf.float32, [None, height, width, num_classes], 'y_pl')
is_training_pl = tf.placeholder(tf.bool, name="is-training_pl")
print('x_pl', x_pl.shape)
print('y_pl', y_pl.shape)

with tf.name_scope('tiramisu'):
    # DOWN SAMPLING
    x = convolution2d(x_pl, num_outputs=k, kernel_size=(3, 3),
                             stride=1, scope="pre-convolution")
    print(x.shape)
    skip1 = dense_block(x, 4)[0]
    
    skip1 = tf.concat([x, skip1], axis=-1)
    x = transition_down(skip1, 4*16, True)
    print(x.shape)

    skip2 = dense_block(x, 5)[0]
    skip2 = tf.concat([x, skip2], axis=-1)
    x = transition_down(skip2, 5*16+4*16, True)
    print(x.shape)

    # BOTTLENECK
    x = dense_block(x, 15)[0]
    print(x.shape)

    # UPSAMPLING
    x = transition_up(x, 15*k)
    x = tf.concat([x, skip2], axis=-1)
    x = dense_block(x, 5)[0]
    print(x.shape)

    x = transition_up(x, 5*k)
    skip_up = tf.concat([x, skip1], axis=-1)
    x = dense_block(skip_up, 4)[0]
    x = tf.concat([x, skip_up], axis=-1)
    print(x.shape)

    # Output layers
    x = convolution2d(x, num_outputs=num_classes, kernel_size=(1, 1),
                             stride=1, scope="post-convolution")
    print(x.shape)
    y = fully_connected(x, num_outputs=num_classes, activation_fn=softmax, scope="SoftMax")
    print(y.shape)

print("Model built")

In [ ]:
with tf.variable_scope('loss'):
    # computing cross entropy per sample
    cross_entropy = -tf.reduce_sum(y_pl * tf.log(y+1e-8), reduction_indices=[1])

    # averaging over samples
    cross_entropy = tf.reduce_mean(cross_entropy)

    
with tf.variable_scope('training'):
    # defining our optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

    # applying the gradients
    train_op = optimizer.minimize(cross_entropy)

    
with tf.variable_scope('performance'):
    # making a one-hot encoded vector of correct (1) and incorrect (0) predictions
    correct_prediction = tf.equal(tf.argmax(y, axis=1), tf.argmax(y_pl, axis=1))

    # averaging the one-hot encoded vector
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [ ]:
#Test the forward pass
x_batch, y_batch = mnist_data.train.next_batch(32)

with tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts)) as sess:
    sess.run(tf.global_variables_initializer())
    y_pred = sess.run(fetches=y, feed_dict={x_pl: x_batch, is_training_pl: True})

assert y_pred.shape == x_batch.shape, "ERROR the output shape is not as expected!" \
        + " Output shape should be " + str(x_batch.shape) + ' but was ' + str(y_pred.shape)

print('Forward pass successful!')

In [ ]:
#Training Loop
batch_size = 10
max_epochs = 10


valid_loss, valid_accuracy = [], []
train_loss, train_accuracy = [], []
test_loss, test_accuracy = [], []


with tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts)) as sess:
    summary_writer = tf.summary.FileWriter('./logs', graph=sess.graph)
    sess.run(tf.global_variables_initializer())
    print('Begin training loop')

    i = 0
    try:
        while mnist_data.train.epochs_completed < max_epochs:
            _train_loss, _train_accuracy = [], []
            
            print("epoch", mnist_data.train.epochs_completed, i)
            i += 1
            ## Run train op
            x_batch, y_batch = mnist_data.train.next_batch(batch_size)
            fetches_train = [train_op, cross_entropy, accuracy]
            feed_dict_train = {x_pl: x_batch, y_pl: x_batch, is_training_pl: True}
            _, _loss, _acc = sess.run(fetches_train, feed_dict_train)
            
            _train_loss.append(_loss)
            _train_accuracy.append(_acc)
            

            ## Compute validation loss and accuracy
            if mnist_data.train.epochs_completed % 1 == 0 \
                    and mnist_data.train._index_in_epoch <= batch_size:
                print("validation")
                train_loss.append(np.mean(_train_loss))
                train_accuracy.append(np.mean(_train_accuracy))

                fetches_valid = [cross_entropy, accuracy]
                
                images = mnist_data.validation.images
                feed_dict_valid = {x_pl: images, y_pl: images, is_training_pl: False}
                _loss, _acc = sess.run(fetches_valid, feed_dict_valid)
                
                valid_loss.append(_loss)
                valid_accuracy.append(_acc)
                print("Epoch {} : Train Loss {:6.3f}, Train acc {:6.3f},  Valid loss {:6.3f},  Valid acc {:6.3f}".format(
                    mnist_data.train.epochs_completed, train_loss[-1], train_accuracy[-1], valid_loss[-1], valid_accuracy[-1]))
        
        
        test_epoch = mnist_data.test.epochs_completed
        while mnist_data.test.epochs_completed == test_epoch:
            x_batch, y_batch = mnist_data.test.next_batch(batch_size)
            feed_dict_test = {x_pl: x_batch, y_pl: x_batch}
            _loss, _acc = sess.run(fetches_valid, feed_dict_test)
            test_loss.append(_loss)
            test_accuracy.append(_acc)
        print('Test Loss {:6.3f}, Test acc {:6.3f}'.format(
                    np.mean(test_loss), np.mean(test_accuracy)))


    except KeyboardInterrupt:
        pass